In [1]:
#######################################
###
### environment setup
###
#######################################

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj, transform, CRS
from shapely.geometry import Polygon, Point
import datetime
from pprint import pprint
import pprint

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# display full
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import geopandas as gpd

In [4]:
import geoplot as gplt

In [5]:
def show_df(df, head_rows=5):
    print(df.info())
    display(df.head(head_rows))

In [6]:
#######################################
###
### identify source data sets
###
#######################################

In [7]:
files = {
    'via_data': '../data_files/via_datadict.csv',
    'tracts' : '../data_files/bexar_county/Bexar_County_Census_Tracts-shp/Bexar_County_Census_Tracts.shp', 
    'block_groups' : '../data_files/bexar_county/Bexar_County_Census_Block_Groups-shp/Bexar_County_Census_Block_Groups.shp', 
    'blocks' : '../data_files/bexar_county/Bexar_County_Census_Blocks-shp/Bexar_County_Census_Blocks.shp', 
    'adherence_2020' : '../data_files/Adherence_2020.csv',
    'bus_otp_2020' : '../data_files/BusOnTimePerformance_2020.csv',
    'bus_fare_box_2020' : '../data_files/bus_fare_fixed.csv',
    'message_logs_2020' : '../data_files/Logged_Messages.csv',
    'service_miles_2020' : '../data_files/TotalServiceMiles_2020.csv',
    'stops_addr_2020' : '../data_files/Stops_LatLong_GeocodeData_2020.csv',
    'stops_201909' : '../data_files/via_201909/stops.txt', 
    'stops_202004' : '../data_files/via_202004/stops.csv', 
    'trips_201909' : '../data_files/via_201909/trips.txt', 
    'trips_202004' : '../data_files/via_202004/trips.csv', 
    'routes_201909' : '../data_files/via_201909/routes.txt', 
    'routes_202004' : '../data_files/via_202004/routes.csv', 
    'stop_times_201909' : '../data_files/via_201909/stop_times.txt', 
    'stop_times_202004' : '../data_files/via_202004/stop_times.csv', 
    'transfers_201909' : '../data_files/via_201909/transfers.txt', 
    'transfers_202004' : '../data_files/via_202004/transfers.csv', 
    'shapes_201909' : '../data_files/via_201909/shapes.txt', 
    'shapes_202004' : '../data_files/via_202004/shapes.csv', 
    'group_stops' : '../data_files/group_stops-shp',
    'acs5_name':'../data_files/census_data/acs5_name.csv',
    'acs5_key':'../data_files/census_data/acs5_key.csv',
    'census_field_names':'../data_files/census_data/census_field_names.csv',
    'census_groups_names':'../data_files/census_data/census_groups_names.csv',

}

In [8]:
via_datadict = pd.read_csv(files['via_data'])
print('*** via_datadict ***\n')
print(via_datadict.info())

via_rename = via_datadict.set_index('via_column')['gotb_column']
print('\n\n*** via_rename ***\n')
print(via_rename.head())

via_datatype = via_datadict.set_index('via_column')['datatype']
print('\n\n*** via_datatype ***\n')
print(via_datatype.head())

gotb_datatype = via_datadict.set_index('gotb_column')['datatype']
print('\n\n*** gotb_datatype ***\n')
print(gotb_datatype.head())

*** via_datadict ***

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   via_column   54 non-null     object 
 1   gotb_column  54 non-null     object 
 2   datatype     54 non-null     object 
 3   description  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 1.8+ KB
None


*** via_rename ***

via_column
ServiceDate           service_date
Routes                       route
Block                  route_block
RouteDirectionName       route_dir
StopNumber                 stop_id
Name: gotb_column, dtype: object


*** via_datatype ***

via_column
ServiceDate           datetime64
Routes                    string
Block                     string
RouteDirectionName        string
StopNumber                string
Name: datatype, dtype: object


*** gotb_datatype ***

gotb_column
service_date    datetime64
route               string
route

In [9]:
#######################################
###
### set up census data
###
#######################################

In [10]:
# gdf_tracts = gpd.read_file(files['tracts']).set_index('OBJECTID')
# gdf_tracts.index.name = 'TRACTID'
# gdf_tracts['density']=gdf_tracts.SUM_POPULA / gdf_tracts.ShapeSTAre.min()
# gdf_tracts = gdf_tracts.to_crs('epsg:4326')
# gdf_tracts = gdf_tracts.astype({'TRACT':'string'})
# show_df(gdf_tracts)

In [11]:
# gdf_groups = gpd.read_file(files['block_groups']).set_index('OBJECTID')
# gdf_groups.index.name = 'GROUPID'
# gdf_groups['density']=gdf_groups.SUM_POPULA / gdf_groups.ShapeSTAre
# gdf_groups = gdf_groups.to_crs('epsg:4326')
# gdf_groups = gdf_groups.astype({'BLOCKGROUP':'string'})
# gdf_groups = gdf_groups.drop(columns=['Shape_STAr','Shape_STLe'])
# show_df(gdf_groups)

In [12]:
# gdf_blocks_cols = [
#     'BLOCK', 'BLOCKGROUP', 'TRACT', 'ID', 'COLORING', 'MCD', 
#     'PLACE', 'VTD', 'CONGRESS', 'LOWERSLD', 'UPPERSLD', 'UNIFSCHOOL',
#     'POPULATION', 'HISPANIC_O', 'NH_WHT', 'NH_BLK', 'NH_ASN', 'NH_OTH',
#     'gecovector', 'ShapeSTAre', 'ShapeSTLen', 'geometry'
# ]

In [13]:
# gdf_blocks = gpd.read_file(files['blocks']).set_index('OBJECTID')
# gdf_blocks.index.name = 'BLOCKOBJ'
# gdf_blocks['NH_OTH'] = (
#     gdf_blocks.POPULATION 
#     - gdf_blocks.HISPANIC_O 
#     - gdf_blocks.NH_WHT 
#     - gdf_blocks.NH_BLK 
#     - gdf_blocks.NH_ASN
# )
# gdf_blocks = gdf_blocks[gdf_blocks_cols]
# gdf_blocks = gdf_blocks.merge(
#     gdf_groups[['BLOCKGROUP','SUM_POPULA']].rename(columns={'SUM_POPULA':'GROUPPOP'}),
#     on='BLOCKGROUP',
#     how='left'
# )
# gdf_blocks['GROUPPCT']=gdf_blocks.POPULATION / gdf_blocks.GROUPPOP
# gdf_blocks = gdf_blocks.merge(
#     gdf_tracts[['TRACT','SUM_POPULA']].rename(columns={'SUM_POPULA':'TRACTPOP'}),
#     on='TRACT',
#     how='left'
# )
# gdf_blocks['TRACTPCT']=gdf_blocks.POPULATION / gdf_blocks.TRACTPOP
# # gdf_blocks['density']=gdf_blocks.POPULATION / gdf_blocks.ShapeSTAre
# gdf_blocks = gdf_blocks.to_crs('epsg:4326')
# gdf_blocks = gdf_blocks.astype({
#     'BLOCK': 'string',
#     'BLOCKGROUP': 'string',
#     'TRACT': 'string',
#     'MCD': 'string',
#     'PLACE': 'string',
#     'VTD': 'string',
#     'CONGRESS': 'string',
#     'LOWERSLD': 'string',
#     'UPPERSLD': 'string',
#     'UNIFSCHOOL': 'string',
# })
# show_df(gdf_blocks)

In [14]:
# gdf_counties = gdf_tracts[['TRACT','SUM_POPULA','ShapeSTAre','geometry']].copy()
# gdf_counties['COUNTY'] = gdf_counties['TRACT'].str[:5]
# gdf_counties.drop(columns='TRACT', inplace=True)
# gdf_counties = gdf_counties.dissolve(by='COUNTY', aggfunc='sum').reset_index()
# gdf_counties.index.name = 'COUNTYID'
# gdf_counties = gdf_counties.astype({'COUNTY':"string"})
# show_df(gdf_counties)

In [15]:
# For simplification, using distance per degree latitude to determine buffer amount. 
# This is slightly inaccurate due to the curvature of the earth. Buffer distances is
# set at 800 meters, which is roughly a half mile.

# meters_per_degree = 111111
# buffer_in_meters = 750
# buffer_in_degrees = buffer_in_meters / meters_per_degree
# buffer_in_degrees

In [16]:
#######################################
###
### set up via datathon data
###
#######################################

In [17]:
#######################################
###
### via datathon 2020
###
### adherence
###
#######################################

In [18]:
def retype_dataframe(df, type_dict):
    retype_dict = {k:v for (k,v) in type_dict.items() if k in df.columns}
    # print(retype_dict)
    string_cols = (k for (k,v) in retype_dict.items() if v == 'string')
    # print(string_cols)
    for col in string_cols:
        df[col] = df[col].apply(str)
    df = df.astype(retype_dict).copy()

    return df

In [19]:
def wrangle_adherence(source_file):

    adherence_stops_cols = ['stop_id','stop_name','deg_lat','deg_lon']
    adherence_stops_drops = adherence_stops_cols.copy()
    adherence_stops_drops.remove('stop_id')
    # print(adherence_stops_drops)
    
    df_adh = pd.read_csv(source_file).rename(columns=via_rename)
    df_adh.route_dir = df_adh.route_dir.str.slice(0,1)
    df_adh.vehicle_id = df_adh.vehicle_id.apply(str)
    df_adh.vehicle_id = df_adh.vehicle_id.apply(lambda x: x.replace('.0',''))
    # print('Check F1: df_adh')
    # print(df_adh.info())
    
    gdf_stops = df_adh[adherence_stops_cols]
    # print('\nCheck F2: df_stops')
    # print(gdf_stops.info())
    
    df_adh = df_adh.drop(columns=adherence_stops_drops)
    df_adh = retype_dataframe(df_adh, gotb_datatype)
    # print('\nCheck F3: df_adh')
    # print(df_adh.info())

    adherence_stops_rename = {
        'deg_lat' : 'stop_lat', 
        'deg_lon' : 'stop_lon',
    }
    
    gdf_stops['stops'] = 1
    gdf_stops.deg_lat.fillna(0, inplace=True)
    gdf_stops.deg_lon.fillna(0, inplace=True)
    gdf_stops.deg_lat = gdf_stops.deg_lat * 10 ** -7
    gdf_stops.deg_lon = gdf_stops.deg_lon * 10 ** -7
    gdf_stops = gdf_stops.groupby(adherence_stops_cols).agg({'stops':['sum']}).reset_index()
    gdf_stops.columns = [col[0] for col in gdf_stops.columns]
    # print(new_cols)
    gdf_stops.rename(columns=adherence_stops_rename, inplace=True)
    gdf_stops['stop_code'] = gdf_stops.stop_id
    gdf_stops.set_index('stop_code', inplace=True)
    gdf_stops = retype_dataframe(gdf_stops, gotb_datatype)
    gdf_stops = gpd.GeoDataFrame(
        gdf_stops,
        geometry = gpd.points_from_xy(gdf_stops.stop_lon, gdf_stops.stop_lat),
        crs={'epsg:4326'}
    )
    # print('\nCheck F4: df_stops')
    # print(gdf_stops.info())
    return df_adh, gdf_stops

In [20]:
df_adherence, gdf_stops_adh = wrangle_adherence(files['adherence_2020'])
# print(adh.type())
# df_adherence = adh[0]
# gdf_stops_adh = adh[1]
print('\ndf_adherence')
show_df(df_adherence)
print('\ngdf_stops_adh')
show_df(gdf_stops_adh)


df_adherence
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358328 entries, 0 to 358327
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   service_date    358328 non-null  datetime64[ns]
 1   route           358328 non-null  string        
 2   route_block     358328 non-null  string        
 3   route_dir       358328 non-null  string        
 4   stop_id         358328 non-null  string        
 5   sched_time_s    358328 non-null  float64       
 6   sched_time_hms  358328 non-null  datetime64[ns]
 7   arr_time_s      329729 non-null  float64       
 8   arr_time_hms    329729 non-null  datetime64[ns]
 9   dep_time_s      329729 non-null  float64       
 10  dep_time_hms    329729 non-null  datetime64[ns]
 11  odometer        329684 non-null  float64       
 12  vehicle_id      358328 non-null  string        
dtypes: datetime64[ns](4), float64(4), string(5)
memory usage: 35.5 MB
None


,service_date,route,route_block,route_dir,stop_id,sched_time_s,sched_time_hms,arr_time_s,arr_time_hms,dep_time_s,dep_time_hms,odometer,vehicle_id
0,2019-09-11,34,002-001,N,GARA,0.0,2020-08-19 03:53:00,13213.0,2020-08-19 03:40:13,13701.0,2020-08-19 03:48:21,0.15,533
1,2019-09-11,34,002-001,N,99166,14940.0,2020-08-19 04:09:00,NaN,NaT,NaN,NaT,NaN,533
2,2019-09-11,34,002-001,N,82589,15540.0,2020-08-19 04:19:00,NaN,NaT,NaN,NaT,NaN,533
3,2019-09-11,34,002-001,N,11726,16020.0,2020-08-19 04:27:00,16081.0,2020-08-19 04:28:01,16094.0,2020-08-19 04:28:14,10.85,533
4,2019-09-11,34,002-001,N,99396,16200.0,2020-08-19 04:30:00,16281.0,2020-08-19 04:31:21,16281.0,2020-08-19 04:31:21,11.28,533



gdf_stops_adh
<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 649 entries, 10013 to GARA
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   stop_id    649 non-null    string  
 1   stop_name  649 non-null    string  
 2   stop_lat   649 non-null    float64 
 3   stop_lon   649 non-null    float64 
 4   stops      649 non-null    int64   
 5   geometry   649 non-null    geometry
dtypes: float64(2), geometry(1), int64(1), string(2)
memory usage: 35.5+ KB
None


,stop_id,stop_name,stop_lat,stop_lon,stops,geometry
stop_code,,,,,,
10013,10013,NORTH STAR TRANSIT CENTER,29.519705,-98.498340,1511,POINT (-98.49834 29.51970)
10019,10019,NORTH STAR TRANSIT CENTER,29.519579,-98.498064,836,POINT (-98.49806 29.51958)
10023,10023,NORTH STAR TRANSIT CENTER,29.519261,-98.498801,575,POINT (-98.49880 29.51926)
10029,10029,NORTH STAR TRANSIT CENTER,29.519446,-98.498229,646,POINT (-98.49823 29.51945)
10033,10033,NORTH STAR TRANSIT CENTER,29.519403,-98.498651,567,POINT (-98.49865 29.51940)


In [21]:
df_adherence.columns

Index(['service_date', 'route', 'route_block', 'route_dir', 'stop_id',
       'sched_time_s', 'sched_time_hms', 'arr_time_s', 'arr_time_hms',
       'dep_time_s', 'dep_time_hms', 'odometer', 'vehicle_id'],
      dtype='object')

In [22]:
adh_routes_group_cols = ['service_date','route_block','sched_time_s','route_dir','vehicle_id','route','stop_id']
adh_routes_data_cols = ['arr_time_s', 'dep_time_s', 'odometer']
adh_routes = df_adherence[adh_routes_group_cols + adh_routes_data_cols]
adh_routes['arr_odometer'] = adh_routes.odometer
adh_routes['dep_odometer'] = adh_routes.odometer
adh_routes['recs'] = 1
adh_routes = adh_routes.groupby(adh_routes_group_cols).agg({'recs':['count'], 'arr_time_s':['min'],'dep_time_s':['max'],'arr_odometer':['min'],'dep_odometer':['max']}).reset_index()
adh_routes.columns = [col[0] for col in adh_routes.columns]
adh_routes = retype_dataframe(adh_routes, gotb_datatype)
adh_routes['sched_time_hms'] = adh_routes.service_date + pd.to_timedelta(adh_routes.sched_time_s,'sec')
adh_routes['arr_time_hms'] = adh_routes.service_date + pd.to_timedelta(adh_routes.arr_time_s,'sec')
adh_routes['dep_time_hms'] = adh_routes.service_date + pd.to_timedelta(adh_routes.dep_time_s,'sec')
adh_routes.drop(columns=['sched_time_s', 'arr_time_s', 'dep_time_s'], inplace=True)
adh_routes_sort_cols = ['service_date','route_block','sched_time_hms','vehicle_id','arr_time_hms','arr_odometer','route_dir']
adh_routes = adh_routes.sort_values(by=adh_routes_sort_cols).reset_index().drop(columns='index')

show_df(adh_routes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357465 entries, 0 to 357464
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   service_date    357465 non-null  datetime64[ns]
 1   route_block     357465 non-null  string        
 2   route_dir       357465 non-null  string        
 3   vehicle_id      357465 non-null  string        
 4   route           357465 non-null  string        
 5   stop_id         357465 non-null  string        
 6   recs            357465 non-null  int64         
 7   arr_odometer    328888 non-null  float64       
 8   dep_odometer    328888 non-null  float64       
 9   sched_time_hms  357465 non-null  datetime64[ns]
 10  arr_time_hms    328933 non-null  datetime64[ns]
 11  dep_time_hms    328933 non-null  datetime64[ns]
dtypes: datetime64[ns](4), float64(2), int64(1), string(5)
memory usage: 32.7 MB
None


,service_date,route_block,route_dir,vehicle_id,route,stop_id,recs,arr_odometer,dep_odometer,sched_time_hms,arr_time_hms,dep_time_hms
0,2019-09-11,002-001,N,533,34,GARA,1,0.15,0.15,2019-09-11 00:00:00,2019-09-11 03:40:13,2019-09-11 03:48:21
1,2019-09-11,002-001,N,533,34,99166,1,NaN,NaN,2019-09-11 04:09:00,NaT,NaT
2,2019-09-11,002-001,N,533,34,82589,1,NaN,NaN,2019-09-11 04:19:00,NaT,NaT
3,2019-09-11,002-001,N,533,34,11726,1,10.85,10.85,2019-09-11 04:27:00,2019-09-11 04:28:01,2019-09-11 04:28:14
4,2019-09-11,002-001,N,533,34,99396,1,11.28,11.28,2019-09-11 04:30:00,2019-09-11 04:31:21,2019-09-11 04:31:21


In [23]:
def generate_run_ids(df):
    #setup
    block_run = 0
    previous = ()
    #for loop - condition then yield
    for row in df.index:
        # cur_df = df.loc[row]
        current=(df.service_date[row],df.vehicle_id[row],df.route[row],df.route_block[row],df.route_dir[row])
        # print(run_stop)
        if previous != current:
            block_run += 1
            run_stop = 1
        if df.stop_id[row] == 'GARA':
            yield row, block_run, None
        else:
            yield row, block_run, run_stop
            run_stop +=1
        previous=current

# adh_routes_group_cols = ['service_date','vehicle_id','route','sched_time_s','route_block','route_dir','stop_id']
# adh_routes_block_cols = adh_routes_group_cols.copy()
# print(adh_routes_block_cols)
# adh_routes_block_cols.remove('sched_time_s')
print(len(adh_routes))
# gen_df = pd.DataFrame(generate_run_ids(adh_routes[adh_routes_group_cols][adh_routes.stop_id != 'GARA']), columns=['idx', 'block_run', 'run_stop'])
gen_df = pd.DataFrame(generate_run_ids(adh_routes), columns=['idx', 'block_run', 'run_stop'])
gen_df = gen_df.merge(
    gen_df[['block_run','run_stop']].groupby(['block_run']).agg('max').rename(columns={'run_stop':'stops'}),
    how='left',
    on='block_run'
    ).set_index('idx')

print()
print(gen_df.head())
print(gen_df.tail())


357465

     block_run  run_stop  stops
idx                            
0            1       NaN    4.0
1            1       1.0    4.0
2            1       2.0    4.0
3            1       3.0    4.0
4            1       4.0    4.0
        block_run  run_stop  stops
idx                               
357460      75019       NaN    2.0
357461      75020       NaN    2.0
357462      75020       1.0    2.0
357463      75020       2.0    2.0
357464      75020       NaN    2.0


In [24]:
gen_df_cols_in_routes = [col for col in gen_df.columns if col in adh_routes.columns]
try:
    adh_routes = adh_routes.drop(columns=gen_df_cols_in_routes)
except:
    print('No drops')
    pass

# try:
#     print(gen_df.info())
adh_routes = adh_routes.join(gen_df, how='left')
# except:
#     print('join failed')
#     pass
# print(adh_routes.info())

adh_routes.dep_time_hms.where(adh_routes.run_stop != adh_routes.stops, None, inplace=True)
# adh_routes=adh_routes.rename(columns={'time_missed':'sched_diff'})
# print(adh_routes[['dep_time_hms']].info())
# adh_routes = adh_routes.astype({'dep_time_hms':'datetime64'})
# print(adh_routes[['dep_time_hms']].info())
adh_routes['time_stopped'] = adh_routes.dep_time_hms - adh_routes.arr_time_hms
adh_routes['sched_diff'] = adh_routes.dep_time_hms - adh_routes.sched_time_hms
adh_routes.sched_diff.where(adh_routes.run_stop != adh_routes.stops, adh_routes.arr_time_hms - adh_routes.sched_time_hms, inplace=True)
adh_routes['err_skip'] = (adh_routes.run_stop.isna() == False) & (adh_routes.arr_time_hms.isna())
adh_routes['err_early'] = (adh_routes.run_stop.isna() == False) & (adh_routes.dep_time_hms.isna() == False) & (adh_routes.sched_diff + pd.Timedelta('00:00:10') < '0') 
adh_routes['err_late'] = (adh_routes.run_stop.isna() == False) & (adh_routes.sched_diff >= '00:05:00') 
adh_routes['on_time'] = adh_routes.err_skip + adh_routes.err_late + adh_routes.err_early == 0
show_df(adh_routes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357465 entries, 0 to 357464
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype          
---  ------          --------------   -----          
 0   service_date    357465 non-null  datetime64[ns] 
 1   route_block     357465 non-null  string         
 2   route_dir       357465 non-null  string         
 3   vehicle_id      357465 non-null  string         
 4   route           357465 non-null  string         
 5   stop_id         357465 non-null  string         
 6   recs            357465 non-null  int64          
 7   arr_odometer    328888 non-null  float64        
 8   dep_odometer    328888 non-null  float64        
 9   sched_time_hms  357465 non-null  datetime64[ns] 
 10  arr_time_hms    328933 non-null  datetime64[ns] 
 11  dep_time_hms    264986 non-null  datetime64[ns] 
 12  block_run       357465 non-null  int64          
 13  run_stop        348993 non-null  float64        
 14  stops           3572

,service_date,route_block,route_dir,vehicle_id,route,stop_id,recs,arr_odometer,dep_odometer,sched_time_hms,arr_time_hms,dep_time_hms,block_run,run_stop,stops,time_stopped,sched_diff,err_skip,err_early,err_late,on_time
0,2019-09-11,002-001,N,533,34,GARA,1,0.15,0.15,2019-09-11 00:00:00,2019-09-11 03:40:13,2019-09-11 03:48:21,1,NaN,4.0,00:08:08,03:48:21,False,False,False,True
1,2019-09-11,002-001,N,533,34,99166,1,NaN,NaN,2019-09-11 04:09:00,NaT,NaT,1,1.0,4.0,NaT,NaT,True,False,False,False
2,2019-09-11,002-001,N,533,34,82589,1,NaN,NaN,2019-09-11 04:19:00,NaT,NaT,1,2.0,4.0,NaT,NaT,True,False,False,False
3,2019-09-11,002-001,N,533,34,11726,1,10.85,10.85,2019-09-11 04:27:00,2019-09-11 04:28:01,2019-09-11 04:28:14,1,3.0,4.0,00:00:13,00:01:14,False,False,False,True
4,2019-09-11,002-001,N,533,34,99396,1,11.28,11.28,2019-09-11 04:30:00,2019-09-11 04:31:21,NaT,1,4.0,4.0,NaT,00:01:21,False,False,False,True


In [25]:
adh_routes[['block_run','run_stop']].groupby(['block_run']).agg('count').rename(columns={'run_stop':'stops'}).head()

,stops
block_run,
1,4
2,7
3,6
4,6
5,6


In [26]:
pd.DataFrame(adh_routes.block_run.value_counts()).rename(columns={'block_run':'stops'}).sort_index().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75020 entries, 1 to 75020
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   stops   75020 non-null  int64
dtypes: int64(1)
memory usage: 1.1 MB


In [27]:
adh_routes[adh_routes.block_run==1]

,service_date,route_block,route_dir,vehicle_id,route,stop_id,recs,arr_odometer,dep_odometer,sched_time_hms,arr_time_hms,dep_time_hms,block_run,run_stop,stops,time_stopped,sched_diff,err_skip,err_early,err_late,on_time
0,2019-09-11,002-001,N,533,34,GARA,1,0.15,0.15,2019-09-11 00:00:00,2019-09-11 03:40:13,2019-09-11 03:48:21,1,NaN,4.0,00:08:08,03:48:21,False,False,False,True
1,2019-09-11,002-001,N,533,34,99166,1,NaN,NaN,2019-09-11 04:09:00,NaT,NaT,1,1.0,4.0,NaT,NaT,True,False,False,False
2,2019-09-11,002-001,N,533,34,82589,1,NaN,NaN,2019-09-11 04:19:00,NaT,NaT,1,2.0,4.0,NaT,NaT,True,False,False,False
3,2019-09-11,002-001,N,533,34,11726,1,10.85,10.85,2019-09-11 04:27:00,2019-09-11 04:28:01,2019-09-11 04:28:14,1,3.0,4.0,00:00:13,00:01:14,False,False,False,True
4,2019-09-11,002-001,N,533,34,99396,1,11.28,11.28,2019-09-11 04:30:00,2019-09-11 04:31:21,NaT,1,4.0,4.0,NaT,00:01:21,False,False,False,True


In [28]:
adh_routes.recs.value_counts()

1    356606
2       857
4         2
Name: recs, dtype: int64

In [29]:
df_adherence.route_dir.value_counts()

S    116631
N    116201
E     62920
W     62576
Name: route_dir, dtype: Int64

In [30]:
df_adherence.vehicle_id.value_counts().head()

nan    1744
525    1374
529    1290
533    1274
585    1273
Name: vehicle_id, dtype: Int64

In [31]:
check_mult = adh_routes[adh_routes.recs>2].iloc[0]
check_mult

service_date      2019-09-12 00:00:00
route_block                   002-008
route_dir                           S
vehicle_id                        500
route                               2
stop_id                         10059
recs                                4
arr_odometer                     57.7
dep_odometer                    57.74
sched_time_hms    2019-09-12 15:59:00
arr_time_hms      2019-09-12 15:56:10
dep_time_hms      2019-09-12 15:59:15
block_run                        8256
run_stop                            2
stops                               6
time_stopped          0 days 00:03:05
sched_diff            0 days 00:00:15
err_skip                        False
err_early                       False
err_late                        False
on_time                          True
Name: 38955, dtype: object

In [32]:
check_mult.route_dir

'S'

In [33]:
df_adherence[
    (df_adherence.route == check_mult.route) & 
    (df_adherence.vehicle_id == check_mult.vehicle_id) & 
    (df_adherence.stop_id == check_mult.stop_id) & 
    (df_adherence.route_dir == check_mult.route_dir) & 
    (df_adherence.sched_time_hms == check_mult.sched_time_hms) 

]

,service_date,route,route_block,route_dir,stop_id,sched_time_s,sched_time_hms,arr_time_s,arr_time_hms,dep_time_s,dep_time_hms,odometer,vehicle_id


In [34]:
df_adherence[
    df_adherence.arr_time_s.isna() != df_adherence.odometer.isna()
].head()

,service_date,route,route_block,route_dir,stop_id,sched_time_s,sched_time_hms,arr_time_s,arr_time_hms,dep_time_s,dep_time_hms,odometer,vehicle_id
11789,2019-09-11,26,020-007,W,11269,24600.0,2020-08-19 06:50:00,24902.0,2020-08-19 06:55:02,24902.0,2020-08-19 06:55:02,NaN,673
12692,2019-09-11,26,020-016,W,11269,56820.0,2020-08-19 15:47:00,56852.0,2020-08-19 15:47:32,56852.0,2020-08-19 15:47:32,NaN,330
12876,2019-09-11,26,020-018,W,11269,53040.0,2020-08-19 14:44:00,53116.0,2020-08-19 14:45:16,53116.0,2020-08-19 14:45:16,NaN,290
50399,2019-09-12,26,020-009,W,11269,59040.0,2020-08-19 16:24:00,59060.0,2020-08-19 16:24:20,59060.0,2020-08-19 16:24:20,NaN,694
50425,2019-09-12,26,020-009,W,11269,72960.0,2020-08-19 20:16:00,73256.0,2020-08-19 20:20:56,73256.0,2020-08-19 20:20:56,NaN,694


In [35]:
df_adherence.arr_time_s.describe()

count    329729.000000
mean      49029.796148
std       17924.176101
min       11681.000000
25%       33741.000000
50%       48911.000000
75%       63287.000000
max       93624.000000
Name: arr_time_s, dtype: float64

In [36]:
df_adherence[df_adherence.arr_time_s>93000]

,service_date,route,route_block,route_dir,stop_id,sched_time_s,sched_time_hms,arr_time_s,arr_time_hms,dep_time_s,dep_time_hms,odometer,vehicle_id
25516,2019-09-11,100,100-010,S,GARA,93360.0,2020-08-19 01:56:00,93108.0,2020-08-19 01:51:48,93108.0,2020-08-19 01:51:48,229.15,966
63858,2019-09-12,100,100-010,S,GARA,93360.0,2020-08-19 01:56:00,93064.0,2020-08-19 01:51:04,93064.0,2020-08-19 01:51:04,228.77,962
102120,2019-09-13,100,100-010,S,10103,93060.0,2020-08-19 01:51:00,93038.0,2020-08-19 01:50:38,93090.0,2020-08-19 01:51:30,225.27,962
102121,2019-09-13,100,100-010,S,GARA,93360.0,2020-08-19 01:56:00,93411.0,2020-08-19 01:56:51,93411.0,2020-08-19 01:56:51,226.57,962
103115,2019-09-13,102,102-010,E,GARA,92460.0,2020-08-19 01:41:00,93624.0,2020-08-19 02:00:24,93624.0,2020-08-19 02:00:24,197.05,982
133668,2019-09-14,100,100-007,S,GARA,93360.0,2020-08-19 01:56:00,93349.0,2020-08-19 01:55:49,93349.0,2020-08-19 01:55:49,229.54,965
159734,2019-09-15,100,100-007,S,GARA,93240.0,2020-08-19 01:54:00,93285.0,2020-08-19 01:54:45,93285.0,2020-08-19 01:54:45,185.18,734
192877,2019-09-16,100,100-010,S,GARA,93360.0,2020-08-19 01:56:00,93228.0,2020-08-19 01:53:48,93228.0,2020-08-19 01:53:48,228.98,966
223726,2019-09-17,289,036-006,S,GARA,91380.0,2020-08-19 01:23:00,93049.0,2020-08-19 01:50:49,93049.0,2020-08-19 01:50:49,251.60,640
231055,2019-09-17,100,100-010,S,GARA,93360.0,2020-08-19 01:56:00,93110.0,2020-08-19 01:51:50,93110.0,2020-08-19 01:51:50,228.84,965


In [37]:
adh_routes[
    (adh_routes.route == check_mult.route) & 
    (adh_routes.vehicle_id == check_mult.vehicle_id) & 
    (adh_routes.service_date == check_mult.service_date) 
    # & 
    # (adh_routes.route_dir == check_mult.route_dir) 
].sort_values(by='sched_time_hms')

,service_date,route_block,route_dir,vehicle_id,route,stop_id,recs,arr_odometer,dep_odometer,sched_time_hms,arr_time_hms,dep_time_hms,block_run,run_stop,stops,time_stopped,sched_diff,err_skip,err_early,err_late,on_time
38944,2019-09-12,002-008,N,500,2,99396,1,42.20,42.20,2019-09-12 14:15:00,2019-09-12 14:16:21,NaT,8252,1.0,1.0,NaT,00:01:21,False,False,False,True
38947,2019-09-12,002-008,N,500,2,56239,2,44.34,44.38,2019-09-12 14:28:00,2019-09-12 14:34:16,2019-09-12 14:36:43,8255,1.0,7.0,00:02:27,00:08:43,False,False,True,False
38948,2019-09-12,002-008,N,500,2,95226,2,46.34,46.37,2019-09-12 14:37:00,2019-09-12 14:48:53,2019-09-12 14:49:59,8255,2.0,7.0,00:01:06,00:12:59,False,False,True,False
38949,2019-09-12,002-008,N,500,2,45506,2,47.84,47.88,2019-09-12 14:44:00,2019-09-12 14:56:03,2019-09-12 14:56:57,8255,3.0,7.0,00:00:54,00:12:57,False,False,True,False
38950,2019-09-12,002-008,N,500,2,10013,2,50.50,50.54,2019-09-12 14:58:00,2019-09-12 15:09:45,2019-09-12 15:09:45,8255,4.0,7.0,00:00:00,00:11:45,False,False,True,False
38951,2019-09-12,002-008,N,500,2,10013,2,50.62,50.68,2019-09-12 15:03:00,2019-09-12 15:09:45,2019-09-12 15:11:44,8255,5.0,7.0,00:01:59,00:08:44,False,False,True,False
38952,2019-09-12,002-008,N,500,2,94709,2,53.53,53.64,2019-09-12 15:18:00,2019-09-12 15:26:03,2019-09-12 15:26:15,8255,6.0,7.0,00:00:12,00:08:15,False,False,True,False
38953,2019-09-12,002-008,N,500,2,94793,2,54.80,54.86,2019-09-12 15:25:00,2019-09-12 15:33:29,NaT,8255,7.0,7.0,NaT,00:08:29,False,False,True,False
38954,2019-09-12,002-008,S,500,2,94793,2,54.92,54.94,2019-09-12 15:43:00,2019-09-12 15:33:29,2019-09-12 15:42:53,8256,1.0,6.0,00:09:24,-1 days +23:59:53,False,False,False,True
38955,2019-09-12,002-008,S,500,2,10059,4,57.70,57.74,2019-09-12 15:59:00,2019-09-12 15:56:10,2019-09-12 15:59:15,8256,2.0,6.0,00:03:05,00:00:15,False,False,False,True


In [38]:
#######################################
###
### block_runs
###
#######################################

In [39]:
adh_block_runs=(adh_routes[adh_routes.run_stop.isna()==False]
    .groupby(by=[
        'block_run'
    ])
    .agg({
        'service_date':['first'],
        'route_block':['first'],
        'route_dir':['first'],
        'vehicle_id':['first'],
        'route':['first'],
        'stops':['first'],
        'recs':['sum',],
        'stop_id':['count',],
        'arr_odometer':['min',],
        'dep_odometer':['max',],
        'sched_time_hms':['min','max'],
        'arr_time_hms':['min',],
        'dep_time_hms':['max',],
        'time_stopped':['sum',],
        # 'sched_diff':['min','max','median'],
        'err_skip':['mean',],
        'err_late':['mean',],
        'err_early':['mean',],
        'on_time':['mean',],
    })
)
# .reset_index().set_index('block_run')
show_df(adh_block_runs)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74834 entries, 1 to 75020
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   (service_date, first)  74834 non-null  datetime64[ns] 
 1   (route_block, first)   74834 non-null  string         
 2   (route_dir, first)     74834 non-null  string         
 3   (vehicle_id, first)    74834 non-null  string         
 4   (route, first)         74834 non-null  string         
 5   (stops, first)         74834 non-null  float64        
 6   (recs, sum)            74834 non-null  int64          
 7   (stop_id, count)       74834 non-null  int64          
 8   (arr_odometer, min)    67912 non-null  float64        
 9   (dep_odometer, max)    67912 non-null  float64        
 10  (sched_time_hms, min)  74834 non-null  datetime64[ns] 
 11  (sched_time_hms, max)  74834 non-null  datetime64[ns] 
 12  (arr_time_hms, min)    67913 non-null  datetim

service_date route_block route_dir vehicle_id route stops recs  \
                 first       first     first      first first first  sum   
block_run                                                                  
1           2019-09-11     002-001         N        533    34   4.0    4   
2           2019-09-11     002-001         N        533     2   7.0    8   
3           2019-09-11     002-001         S        533     2   6.0    7   
4           2019-09-11     002-001         S        533    34   6.0    6   
5           2019-09-11     002-001         N        533    34   6.0    6   

          stop_id arr_odometer dep_odometer      sched_time_hms  \
            count          min          max                 min   
block_run                                                         
1               4        10.85        11.28 2019-09-11 04:09:00   
2               7        11.36        23.41 2019-09-11 04:30:00   
3               6        23.49        32.42 2019-09-11 05:26:00   
4               6        34.65        43.75 2019-09-11 06:04:00   
5               6        43.82        52.99 2019-09-11 07:02:00   

                                     arr_time_hms        dep_time_hms  \
                          max                 min                 max   
block_run                                                               
1         2019-09-11 04:30:00 2019-09-11 04:28:01 2019-09-11 04:28:14   
2         2019-09-11 05:21:00 2019-09-11 04:31:21 2019-09-11 05:15:34   
3         2019-09-11 06:04:00 2019-09-11 05:19:44 2019-09-11 06:04:12   
4         2019-09-11 06:49:00 2019-09-11 06:15:15 2019-09-11 06:43:24   
5         2019-09-11 07:45:00 2019-09-11 06:53:20 2019-09-11 07:48:00   

          time_stopped  err_skip  err_late err_early   on_time  
                   sum      mean      mean      mean      mean  
block_run                                                       
1             00:00:13  0.500000  0.000000  0.000000  0.500000  
2             00:08:25  0.000000  0.000000  0.142857  0.857143  
3             00:11:43  0.166667  0.166667  0.000000  0.666667  
4             00:03:17  0.166667  0.500000  0.000000  0.333333  
5             00:13:03  0.333333  0.500000  0.000000  0.166667

In [40]:
adh_routes[(adh_routes.vehicle_id=='533') & (adh_routes.service_date=='2019-09-11')]

,service_date,route_block,route_dir,vehicle_id,route,stop_id,recs,arr_odometer,dep_odometer,sched_time_hms,arr_time_hms,dep_time_hms,block_run,run_stop,stops,time_stopped,sched_diff,err_skip,err_early,err_late,on_time
0,2019-09-11,002-001,N,533,34,GARA,1,0.15,0.15,2019-09-11 00:00:00,2019-09-11 03:40:13,2019-09-11 03:48:21,1,NaN,4.0,00:08:08,03:48:21,False,False,False,True
1,2019-09-11,002-001,N,533,34,99166,1,NaN,NaN,2019-09-11 04:09:00,NaT,NaT,1,1.0,4.0,NaT,NaT,True,False,False,False
2,2019-09-11,002-001,N,533,34,82589,1,NaN,NaN,2019-09-11 04:19:00,NaT,NaT,1,2.0,4.0,NaT,NaT,True,False,False,False
3,2019-09-11,002-001,N,533,34,11726,1,10.85,10.85,2019-09-11 04:27:00,2019-09-11 04:28:01,2019-09-11 04:28:14,1,3.0,4.0,00:00:13,00:01:14,False,False,False,True
4,2019-09-11,002-001,N,533,34,99396,1,11.28,11.28,2019-09-11 04:30:00,2019-09-11 04:31:21,NaT,1,4.0,4.0,NaT,00:01:21,False,False,False,True
5,2019-09-11,002-001,N,533,2,99396,1,11.36,11.36,2019-09-11 04:30:00,2019-09-11 04:31:21,2019-09-11 04:32:02,2,1.0,7.0,00:00:41,00:02:02,False,False,False,True
6,2019-09-11,002-001,N,533,2,56239,1,13.19,13.19,2019-09-11 04:39:00,2019-09-11 04:41:47,2019-09-11 04:42:55,2,2.0,7.0,00:01:08,00:03:55,False,False,False,True
7,2019-09-11,002-001,N,533,2,95226,1,14.94,14.94,2019-09-11 04:47:00,2019-09-11 04:48:26,2019-09-11 04:48:37,2,3.0,7.0,00:00:11,00:01:37,False,False,False,True
8,2019-09-11,002-001,N,533,2,45506,1,16.44,16.44,2019-09-11 04:53:00,2019-09-11 04:52:24,2019-09-11 04:52:55,2,4.0,7.0,00:00:31,-1 days +23:59:55,False,False,False,True
9,2019-09-11,002-001,N,533,2,10013,2,19.10,19.25,2019-09-11 05:04:00,2019-09-11 05:01:29,2019-09-11 05:05:08,2,5.0,7.0,00:03:39,00:01:08,False,False,False,True


In [41]:
#######################################
###
### stops
###
#######################################

In [42]:
# def wrangle_stops(source_file, actv_date='20_04'):
    
#     keep_cols = [
#         'stop_id', 'stop_name', 'stop_lat', 'stop_lon'
#     ]
    
#     gdf = pd.read_csv(source_file)
#     gdf.stop_code = gdf.stop_code.apply(str)
#     gdf.stop_id = gdf.stop_id.apply(str)
#     gdf = gdf.astype({'stop_code':'string', 'stop_id':'string', 'stop_name':'string'})
#     gdf.set_index('stop_code', inplace=True)
#     gdf = gdf[keep_cols]
#     gdf = gpd.GeoDataFrame(
#         gdf,
#         geometry = gpd.points_from_xy(gdf.stop_lon, gdf.stop_lat),
#         crs={'epsg:4326'}
#     )
#     # gdf.stop_id = gdf.stop_id.apply(str)
#     gdf['actv_' + str(actv_date)] = 1
#     return gdf


In [43]:
#######################################
###
### stops
###
### stops adh
###
#######################################

In [44]:
# gdf_stops_adh['adh_stop'] = 1
# gdf_stops_adh = gdf_stops_adh.astype({'adh_stop':'int64'})
# show_df(gdf_stops_adh)


In [45]:
# gdf_stops_adh[gdf_stops_adh.stop_lat==0].stop_id.to_list()

In [46]:
# gdf_stops = gdf_stops_adh[gdf_stops_adh.stop_lat != False].drop(columns='stops')
# show_df(gdf_stops)

In [47]:
#######################################
###
### stops
###
### stops 201909
###
#######################################

In [48]:
# gdf_stops_201909 = wrangle_stops(files['stops_201909'], actv_date='19_09')
# gdf_stops_201909 = gdf_stops_201909.join(gdf_stops_adh[['adh_stop']], how='left')
# gdf_stops_201909 = gdf_stops_201909.join(gdf_stops[['adh_stop']], how='left', rsuffix='2')

# gdf_stops_201909.adh_stop.fillna(0, inplace=True)
# gdf_stops_201909.adh_stop2.fillna(0, inplace=True)

# print('*** gdf_stops_201909 ***\n')
# print(gdf_stops_201909.info())
# print('\nadh stops: ', gdf_stops_201909.adh_stop.sum())
# print('\nadh2 stops:', gdf_stops_201909.adh_stop2.sum())
# gdf_stops_201909.head()

In [49]:
#######################################
###
### stops
###
### stops 200404
###
#######################################

In [50]:
# gdf_stops = gdf_stops.join(gdf_stops_201909['actv_19_09'], how='inner')
# show_df(gdf_stops)

In [51]:
# gdf_stops.actv_19_09.sum()
# gdf_stops_19_append = gdf_stops_201909[gdf_stops_201909.adh_stop2==0]
# gdf_stops_19_append = gdf_stops_19_append[gdf_stops.columns]
# show_df(gdf_stops_19_append)

In [52]:
# gdf_stops = gdf_stops.append(gdf_stops_19_append, verify_integrity=True)


In [53]:
# gdf_stops=gdf_stops.astype({'adh_stop':'int64'})
# print(gdf_stops.adh_stop.sum())
# show_df(gdf_stops)

In [54]:
# gdf_stops_202004 = wrangle_stops(files['stops_202004'], actv_date='20_04')

# print(print(gdf_stops_202004.)
# gdf_stops_202004.head()info())

In [55]:
# gdf_stops = gdf_stops_202004.join(gdf_stops_201909[['actv_19_09']], how='left')
# gdf_stops.actv_19_09 = gdf_stops.actv_19_09.fillna(0)
# gdf_stops.info()

In [56]:
# adh_join = gdf_stops_adh.join(gdf_stops_201909[['stop_id']], how='left', on='stop_id', rsuffix='_all')
# adh_join.info()

In [57]:
# gdf_stops_202004[gdf_stops_202004.stop_id == '98013']

In [58]:
# adherence_stops_cols = ['StopNumber','Location','Latitude','Longitude','geometry']

In [59]:
#######################################
###
### stops - all
###
#######################################

In [60]:
# adh_stops_merge = gdf_stops_adh[['geometry', 'stop_id']].rename(columns={'geometry': 'geometry_adh', 'stop_id': 'stop_id_adh'})
# all_stops = gdf_stops_202004[['geometry','stop_id']].join(gdf_stops_201909[['geometry','stop_id']], how='outer', lsuffix='_20', rsuffix='_19')
# all_stops = all_stops.join(adh_stops_merge, how='outer')

# all_stops['actv_20_04'] = all_stops.stop_id_20.isna()==False
# all_stops['actv_19_09'] = all_stops.stop_id_19.isna()==False
# all_stops['actv_adh'] = all_stops.stop_id_adh.isna() == False
# all_stops['same_actv'] = all_stops.actv_20_04.astype('int32') + all_stops.actv_19_09.astype('int32') + all_stops.actv_adh.astype('int32')

# # all_stops['same_name'] = all_stops.stop_name_20 == all_stops.stop_name_19
# all_stops['same_geom'] = all_stops.geometry_20 == all_stops.geometry_19
# all_stops['geom_adh_20'] = all_stops.geometry_20 == all_stops.geometry_adh
# all_stops['geom_adh_19'] = all_stops.geometry_19 == all_stops.geometry_adh
# all_stops['same_lat'] = all_stops.stop_lat_20 == all_stops.stop_lat_19
# all_stops['same_lon'] = all_stops.stop_lon_20 == all_stops.stop_lon_19
# all_stops['same_wheel'] = all_stops.wheelchair_boarding_20 == all_stops.wheelchair_boarding_19
# all_stops.info()

In [61]:
# adh_stops_chk = all_stops[all_stops.actv_adh]
# adh_stops_chk[adh_stops_chk.same_actv == 1]
# adh_stops_chk[all_stops.columns]
# adh_stops_chk[adh_stops_chk.same_actv == 1]
# adh_mask = adh_stops_chk.same_actv < 3
# adh_stops_chk[adh_mask]
# adh_stops_chk.index

In [62]:
# print(adh_stops_merge.index.to_list()[:20])
# print(all_stops.index.to_list()[:20])


In [63]:
# same_stops = all_stops[all_stops['same_actv']]
# same_stops.info()

In [64]:
# same_stops[['same_geom']].sum()

In [65]:
# all_stops[all_stops.same_geom == False]

In [66]:
# gdf_stops_202004_blocks[['BLOCK','geometry']].info()

In [67]:
# def buffer_stops(stops_gdf, geo_col='geometry', buffer=.01):

#     gdf = stops_gdf.copy()
    
#     gdf['geometry'] = gdf.buffer(buffer)
#     gdf = gdf.set_geometry('geometry')

#     return gdf

In [68]:
# gdf_stops_buff = buffer_stops(gdf_stops[['stop_id','geometry']], buffer=buffer_in_degrees)
# show_df(gdf_stops_buff)

In [69]:
# def match_stops(stops_gdf, area_gdf, stops_geo='geometry', area_geo='geometry'):
#     sidx_name = stops_gdf.index.name
#     aidx_name = area_gdf.index.name
#     sidx_geo = sidx_name + '_geo'
#     aidx_geo = aidx_name + 'geo'
#     sgdf = stops_gdf[[stops_geo]].copy().reset_index()
#     agdf = area_gdf[[area_geo]].copy().reset_index()
    
#     return sidx_name

# check_match = match_stops(area_gdf=gdf_blocks[['geometry']], stops_gdf=gdf_stops_202004_buff)
# check_match

In [70]:
# # gdf_stops_blocks = gpd.sjoin(gdf_blocks[['BLOCK','geometry']], gdf_stops_202004_buff, how='inner', op='intersects')
# gdf_stops_fips = gpd.sjoin(gdf_blocks[['BLOCK','BLOCKGROUP','TRACT','geometry']], gdf_stops_buff, how='inner', op='intersects')
# gdf_stops_fips = gdf_stops_fips[['BLOCK','BLOCKGROUP','TRACT','stop_id','geometry']]
# # gdf_stops_blocks = gdf_stops_blocks.rename(columns={'index_right':'stop_id'})
# show_df(gdf_stops_fips)

In [71]:
# block_stops = pd.DataFrame(gdf_stops_fips.BLOCK.value_counts()).rename(columns={'BLOCK':'stop_count'})
# block_stops.index.name = 'BLOCK'
# block_stops = block_stops.merge(gdf_blocks, on=['BLOCK'])
# print(block_stops.info())
# print('\nstop count:',block_stops.stop_count.sum())
# # print(block_stops.head())
# # print(block_stops.tail())
# block_stops.head().append(block_stops.tail())


In [72]:
# gdf_groups_stops = gdf_stops_fips[['BLOCKGROUP','geometry']].dissolve(by='BLOCKGROUP', aggfunc='count')
# show_df(gdf_groups_stops)

In [73]:
# groups_stops = (gdf_stops_fips
#     .groupby(['BLOCKGROUP','stop_id'])
#     .agg({'TRACT':'count'})
#     .reset_index()
#     .groupby('BLOCKGROUP')
#     .agg({'stop_id':'count'})
#     .rename(columns={'stop_id':'stops'})
# )
# show_df(groups_stops)

# group_stops = group_stops.group_by([BLOCKGROUP])

In [74]:
# groups_stops_col_drops=['index','stops']
# gdf_groups_stops = gdf_groups_stops.drop(columns=[col for col in groups_stops_col_drops if col in gdf_groups_stops.columns])
# gdf_groups_stops = gdf_groups_stops.merge(groups_stops, on='BLOCKGROUP', how='left')
# # gdf_groups_stops.index.name='OBJECTID'
# show_df(gdf_groups_stops)

In [75]:
# read_shp = gpd.read_file(files['group_stops'])
# show_df(read_shp)

In [76]:
# gdf_groups_stops.to_file(files['group_stops'])

In [77]:
# fig, ax = plt.subplots(figsize=(20,20), facecolor='black')
# ax.set_aspect('equal')
# # ax.set_facecolor('#ff474c')
# axbase = gplt.polyplot(gdf_counties, ax=ax, facecolor='lightgrey', edgecolor='red', linewidth=5)
# gplt.polyplot(gdf_groups, ax=ax, facecolor='None', edgecolor='red', linewidth=.5)
# # gdf_counties.plot()
# read_shp.plot(column='stops', ax=ax, cmap='Reds', alpha=1.75)
# plt.savefig('block_route_frequency.png')

In [78]:
# group_stops = pd.DataFrame(gdf_stops_fips.BLOCKGROUP.value_counts()).rename(columns={'BLOCKGROUP':'stop_count'})
# group_stops.index.name = 'BLOCKGROUP'
# show_df(group_stops)

In [79]:
# stop_blocks = gdf_stops_blocks.stop_id.value_counts()
# len(stop_blocks)

In [80]:
# gdf_stops_blocks[['BLOCK','geometry']].head()

In [81]:
# def wrangle_stops2(source_file, actv_date='20_04'):
    
#     keep_cols = [
#         'stop_id', 'stop_name', 'stop_lat', 'stop_lon'
#     ]
    
#     gdf = pd.read_csv(source_file)
#     gdf.stop_code = gdf.stop_code.apply(str)
#     gdf.stop_id = gdf.stop_id.apply(str)
#     gdf = gdf.astype({'stop_code':'string', 'stop_id':'string', 'stop_name':'string'})
#     gdf.set_index('stop_code', inplace=True)
#     gdf = gdf[keep_cols]
#     gdf = gpd.GeoDataFrame(
#         gdf,
#         geometry = gpd.points_from_xy(gdf.stop_lon, gdf.stop_lat),
#         crs={'epsg:4326'}
#     )
#     # gdf.stop_id = gdf.stop_id.apply(str)
#     gdf['actv_' + str(actv_date)] = 1
#     return gdf


In [82]:
# gdf_stops_201909 = wrangle_stops(files['stops_201909'], actv_date='19_09')
# gdf_stops_201909 = gdf_stops_201909.join(gdf_stops_adh[['adh_stop']], how='left')
# gdf_stops_201909 = gdf_stops_201909.join(gdf_stops[['adh_stop']], how='left', rsuffix='2')

# gdf_stops_201909.adh_stop.fillna(0, inplace=True)
# gdf_stops_201909.adh_stop2.fillna(0, inplace=True)

# print('*** gdf_stops_201909 ***\n')
# print(gdf_stops_201909.info())
# print('\nadh stops: ', gdf_stops_201909.adh_stop.sum())
# print('\nadh2 stops:', gdf_stops_201909.adh_stop2.sum())
# gdf_stops_201909.head()

In [83]:
# chktrips=pd.read_csv(files['trips_201909'])
# show_df(chktrips)

In [84]:
# chkroutetrips=pd.DataFrame(chktrips.route_id.value_counts())
# show_df(chkroutetrips)

In [85]:
# chkrouteservice=pd.DataFrame(chktrips.service_id.value_counts())
# show_df(chkrouteservice)

In [86]:
# weekday_trip_ids = chktrips[chktrips.service_id == 1][['trip_id','route_id']]
# show_df(weekday_trip_ids)

In [87]:
# ''
# chkroutes=pd.read_csv(files['routes_201909'])
# show_df(chkroutes)


In [88]:
# ''
# chktimes=pd.read_csv(files['stop_times_201909'])
# show_df(chktimes)

In [89]:
# chktimestops = pd.DataFrame(chktimes.stop_id.value_counts())
# show_df(chktimestops)

In [90]:
# chktimetrips = pd.DataFrame(chktimes.trip_id.value_counts())
# show_df(chktimetrips)

In [91]:
# trip_stops = chktimes[['trip_id','stop_id']]
# show_df(trip_stops)

In [92]:
# weekday_trip_stops = trip_stops.merge(weekday_trip_ids, how='inner', on='trip_id', )
# weekday_trip_stops['is_stop']=1
# weekday_trip_stops.stop_id = weekday_trip_stops.stop_id.apply(str)
# weekday_trip_stops = weekday_trip_stops.astype({'stop_id':'string'}
# )
# show_df(weekday_trip_stops)

In [93]:
# groups_stop_ids = (gdf_stops_fips
#     .groupby(['BLOCKGROUP','stop_id'])
#     .agg({'TRACT':'count'})
#     .reset_index())
# show_df(groups_stop_ids)

In [94]:
# trip_group_stops = groups_stop_ids.merge(weekday_trip_stops, how='inner', on='stop_id')
# show_df(trip_group_stops)

In [95]:
# trip_group_stops.trip_id.value_counts().count()

In [96]:
# group_trips = (trip_group_stops
#     .groupby(['BLOCKGROUP','trip_id']).agg({'stop_id':'count'})
#     .reset_index()
#     .groupby('BLOCKGROUP').agg({'trip_id':'count'})
#     .rename(columns={'trip_id':'trips'})
# )
# show_df(group_trips)

In [97]:
# group_trips.to_csv('group_trips.csv')

In [98]:
# gdf_group_cents = gdf_groups_stops.centroid

# print(type(gdf_group_cents))
# print(gdf_group_cents.head())
# # show_df(gdf_group_cents)

In [99]:
# gdf_group_trips_c = gpd.GeoDataFrame(gdf_group_cents,columns=['geometry'])
# gdf_group_trips_c = gdf_group_trips_c.join(group_trips).reset_index()
# show_df(gdf_group_trips_c)

In [100]:
# gdf_group_trips_c.to_file('../data_files/gdf_group_trips_c.geojson',driver='GeoJSON')

In [101]:
# # gdf_group_trips_b = gdf_group_trips_c.drop(columns='geometry').merge(gdf_groups_stops, on='OBJECTID')
# gdf_group_trips_b = gdf_group_trips_c.drop(columns='geometry').merge(gdf_groups_stops, on='BLOCKGROUP')
# show_df(gdf_group_trips_b)

In [102]:
# fig, ax = plt.subplots(figsize=(20,20), facecolor='black')
# ax.set_aspect('equal')
# # ax.set_facecolor('#ff474c')
# axbase = gplt.polyplot(gdf_counties, ax=ax, facecolor='lightgrey', edgecolor='red', linewidth=5)
# gplt.polyplot(gdf_groups, ax=ax, facecolor='None', edgecolor='red', linewidth=.5)
# # gdf_counties.plot()
# gdf_group_trips_b.plot(column='trips', ax=ax, cmap='Reds', alpha=1.75)
# plt.savefig('trips_by_group.png')

In [103]:
#######################################
###
### via datathon 2020
###
### on time performance
###
#######################################

In [104]:
# df_otp = pd.read_csv(files['bus_otp_2020'])
# df_otp = df_otp.astype({'ServiceDateTime':'datetime64','Route':'category','RouteName':'string'})
# df_otp.OTP = df_otp.OTP / 100
# show_df(df_otp)

In [105]:
# df_otp.describe().T

In [106]:
# df_otp.columns

In [107]:
# df_otp.groupby(by='ServiceDateTime').count().head(14)

In [108]:
#######################################
###
### via datathon 2020
###
### fare box activity
###
#######################################

In [109]:
# df_fare = pd.read_csv(files['bus_fare_box_2020'], index_col=0)
# show_df(df_fare)

In [110]:
# df_fare.columns

In [111]:
#######################################
###
### via datathon 2020
###
### message logs
###
#######################################

In [112]:
# df_logs = pd.read_csv(files['message_logs_2020'])
# show_df(df_logs)

In [113]:
# df_logs.columns

In [114]:
# df_logs.MESSAGE_TYPE_TEXT.value_counts()

In [115]:
# df_logs.ADHERENCE.value_counts().describe()

In [116]:
# vehicle_ids = df_logs.VehicleNumber.value_counts()
# print(len(vehicle_ids))
# vehicle_ids.head(20)

In [117]:
# df_logs[df_logs.VehicleNumber==450].head(20)

In [118]:
#######################################
###
### via datathon 2020
###
### service miles
###
#######################################

In [119]:
# df_svc_miles = pd.read_csv(files['service_miles_2020'])
# show_df(df_svc_miles)

In [120]:
# df_svc_miles.columns


In [121]:
#######################################
###
### via datathon 2020
###
### stop addresses
###
#######################################

In [122]:
# df_stop_addrs = pd.read_csv(files['stops_addr_2020'])
# show_df(df_stop_addrs)

In [123]:
# df_stop_addrs.columns

In [124]:
# block_pop = gdf_blocks[['BLOCK','BLOCKGROUP','POPULATION','GROUPPCT']].merge(block_stops.drop(columns=['BLOCKGROUP','POPULATION','GROUPPCT']), on='BLOCK', how='left')
# block_pop.stop_count.fillna(0,inplace=True)
# show_df(block_pop)

In [125]:
# block_pop_all = block_pop.POPULATION.sum()
# block_pop_stop = block_pop.POPULATION[block_pop.stop_count > 0].sum()
# block_pop_not = block_pop.POPULATION[block_pop.stop_count == 0].sum()
# block_pop_stop_pct = block_pop_stop / block_pop_all
# block_pop_not_pct = block_pop_not / block_pop_all

# print('all:   ',block_pop_all)
# print('stop:  ',block_pop_stop)
# print('not:   ',block_pop_not)
# print('stop%: ',block_pop_stop_pct)
# print('not%:  ',block_pop_not_pct)

In [126]:
# rdf_name=pd.read_csv(files['acs5_name']).set_index('block_group')
# show_df(rdf_name)